<a href="https://colab.research.google.com/github/Colin-debug236/Lim-Tian-Sheng-Colin-580ZDDDD-AI-Principles-Mini-Project/blob/main/Lim_Tian_Sheng_Colin_(580ZDDDD)Battleship_Game_(Min_Max_Algorithm).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## How to Play the Battleship Game
#1. Run all cells in the notebook by selecting **Runtime > Run All**.
#2. Follow the on-screen prompts to make your moves.
#3. Enjoy!

import random

# Constants
GRID_SIZE = 10
SHIPS = [
    ("Carrier", 4),
    ("Battleship", 3),
    ("Destroyer", 2),
    ("Submarine", 1)
]

# Helper functions
def create_grid():
    return [['~'] * GRID_SIZE for _ in range(GRID_SIZE)]

def display_grid(grid):
    print("  " + " ".join(map(str, range(GRID_SIZE))))
    for i, row in enumerate(grid):
        print(f"{i} " + " ".join(row))

def is_valid_placement(grid, row, col, size, direction):
    if direction == 'H':
        if col + size > GRID_SIZE:
            return False
        for i in range(size):
            if grid[row][col + i] != '~':
                return False
    elif direction == 'V':
        if row + size > GRID_SIZE:
            return False
        for i in range(size):
            if grid[row + i][col] != '~':
                return False
    return True

def place_ship(grid, row, col, size, direction):
    if direction == 'H':
        for i in range(size):
            grid[row][col + i] = 'S'
    elif direction == 'V':
        for i in range(size):
            grid[row + i][col] = 'S'

# AI Player
class AI:
    def __init__(self, difficulty='easy'):
        self.difficulty = difficulty
        self.memory = []

    def random_attack(self, grid):
        while True:
            row = random.randint(0, GRID_SIZE - 1)
            col = random.randint(0, GRID_SIZE - 1)
            if grid[row][col] == '~':
                return row, col

    def minimax_attack(self, grid):
        for row in range(GRID_SIZE):
            for col in range(GRID_SIZE):
                if grid[row][col] == 'X':
                    for dr, dc in [(0, 1), (0, -1), (1, 0), (-1, 0)]:
                        r, c = row + dr, col + dc
                        if 0 <= r < GRID_SIZE and 0 <= c < GRID_SIZE and grid[r][c] == '~':
                            return r, c
        return self.random_attack(grid)

    def make_move(self, grid):
        if self.difficulty == 'easy':
            return self.random_attack(grid)
        elif self.difficulty == 'medium':
            if random.random() < 0.3:
                return self.minimax_attack(grid)
            else:
                return self.random_attack(grid)
        elif self.difficulty == 'hard':
            return self.minimax_attack(grid)

    def place_ships(self):
        grid = create_grid()
        for ship_name, ship_size in SHIPS:
            while True:
                row, col = random.randint(0, GRID_SIZE - 1), random.randint(0, GRID_SIZE - 1)
                direction = random.choice(['H', 'V'])
                if is_valid_placement(grid, row, col, ship_size, direction):
                    place_ship(grid, row, col, ship_size, direction)
                    break
        return grid

# Game class
class BattleshipGame:
    def __init__(self, ai_difficulty='easy'):
        self.player_grid = create_grid()
        self.opponent_grid = create_grid()
        self.player_hits = create_grid()
        self.opponent_hits = create_grid()
        self.ships = SHIPS
        self.mode = 'AI'
        self.ai = AI(ai_difficulty)

    def place_player_ships(self):
        for ship_name, ship_size in SHIPS:
            display_grid(self.player_grid)
            while True:
                print(f"Place your {ship_name} (size {ship_size})")
                choice = input("Enter row,col,direction (e.g., 3,4,H) or type 'auto' to place automatically: ").strip().lower()

                if choice == 'auto':
                    while True:
                        row, col = random.randint(0, GRID_SIZE - 1), random.randint(0, GRID_SIZE - 1)
                        direction = random.choice(['H', 'V'])
                        if is_valid_placement(self.player_grid, row, col, ship_size, direction):
                            place_ship(self.player_grid, row, col, ship_size, direction)
                            break
                    print(f"{ship_name} placed automatically.")
                    display_grid(self.player_grid)
                    break

                try:
                    row, col, direction = choice.split(',')
                    row, col = int(row), int(col)
                    direction = direction.upper()
                    if direction in ['H', 'V'] and is_valid_placement(self.player_grid, row, col, ship_size, direction):
                        place_ship(self.player_grid, row, col, ship_size, direction)
                        display_grid(self.player_grid)
                        break
                    else:
                        print("Invalid placement. Try again.")
                except (ValueError, IndexError):
                    print("Invalid input format. Use 'row,col,direction' or 'auto'.")

    def player_turn(self):
        display_grid(self.player_hits)
        while True:
            try:
                row = int(input("Enter row to attack: "))
                col = int(input("Enter column to attack: "))
                if self.opponent_hits[row][col] == '~':
                    if self.opponent_grid[row][col] == 'S':
                        print("Hit!")
                        self.opponent_hits[row][col] = 'X'
                    else:
                        print("Miss!")
                        self.opponent_hits[row][col] = 'O'
                    break
                else:
                    print("Already attacked there. Try again.")
            except (ValueError, IndexError):
                print("Invalid input. Try again.")

    def opponent_turn(self):
        row, col = self.ai.make_move(self.player_hits)
        print(f"AI attacks {row}, {col}")
        if self.player_grid[row][col] == 'S':
            print("AI hit your ship!")
            self.player_hits[row][col] = 'X'
        else:
            print("AI missed!")
            self.player_hits[row][col] = 'O'

    def check_winner(self, grid):
        return all(cell != 'S' for row in grid for cell in row)

    def start_game(self):
        self.place_player_ships()
        print("AI is placing ships...")
        self.opponent_grid = self.ai.place_ships()

        while True:
            self.player_turn()
            if self.check_winner(self.opponent_grid):
                print("Player wins!")
                break
            self.opponent_turn()
            if self.check_winner(self.player_grid):
                print("AI wins!")
                break

# Main function
def main():
    print("Welcome to Battleship by Colin Lim 580ZDDDD!")
    difficulty = input("Choose AI difficulty (easy, medium, hard): ").lower()
    game = BattleshipGame(difficulty)
    game.start_game()

if __name__ == "__main__":
    main()


Welcome to Battleship by Colin Lim 580ZDDDD!
